In [ ]:
#!/usr/bin/env python
# coding: utf-8


In [ ]:
# In[1]:


In [ ]:

# download the corpus
get_ipython().system('wget http://www.manythings.org/anki/fra-eng.zip')


In [ ]:

# In[3]:


In [ ]:

# unzip the file
get_ipython().system('unzip fra-eng.zip')


In [ ]:

# In[5]:


In [ ]:

import string
import re
import pickle as pkl
import numpy as np
from unicodedata import normalize


In [ ]:
# Load the file to preprocess
def load_file(filename):
    file = open(filename, mode='rt', encoding='utf-8')
    text = file.read()
    file.close()
    return text


In [ ]:
# Split the text into sentences
def to_pair(text):
    sentences = text.strip().split('\n')
    pairs = [s.strip().split('\t') for s in sentences]
    return pairs


In [ ]:
# Clean a list of lines
def clean_pairs(lines):
    cleaned = list()
    # Regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            line = line.split()
            line = [word.lower() for word in line]
            line = [word.translate(table) for word in line]
            line = [re_print.sub('', word) for word in line]
            # Remove numeric chrs
            line = [w for w in line if w.isalpha()]
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return np.array(cleaned)


In [ ]:
# Save the cleaned data to the given filename
def save_data(sentences, filename):
    pkl.dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)


In [ ]:
filename = 'fra.txt'
file = load_file(filename)
pairs = to_pair(file)
print(pairs)
clean_pairs = clean_pairs(pairs)
save_data(clean_pairs, 'english-french.pkl')


In [ ]:
# Checking the cleaned data
for i in range(100):
  print('[%s] => [%s]' % (clean_pairs[i, 0], clean_pairs[i, 1]))


In [ ]:

# In[6]:


In [ ]:

import pickle as pkl
from numpy import random


In [ ]:
def load_clean_data(filename):
    file = open(filename, 'rb')
    return pkl.load(file)


In [ ]:
def save_clean_data(sentences, filename):
    pkl.dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)


In [ ]:

raw_data = load_clean_data('english-french.pkl')


In [ ]:
dataset = raw_data[:10000, :]
random.shuffle(dataset)


In [ ]:
train_set = dataset[:9000, :]
test_set = dataset[9000:, :]


In [ ]:
save_clean_data(dataset, 'english-french-both.pkl')
save_clean_data(train_set, 'english-french-train.pkl')
save_clean_data(test_set, 'english-french-test.pkl')


In [ ]:

# In[7]:


In [ ]:

# Training Encoder-Decoder model to represent word embeddings and finally
# save the trained model as 'model.h5'


In [ ]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint


In [ ]:

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))


In [ ]:

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer


In [ ]:

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)


In [ ]:

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X


In [ ]:

# one hot encode target sequence
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y


In [ ]:

# define NMT model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
	model = Sequential()
	model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
	model.add(LSTM(n_units))
	model.add(RepeatVector(tar_timesteps))
	model.add(LSTM(n_units, return_sequences=True))
	model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
	return model


In [ ]:

# load datasets
dataset = load_clean_sentences('english-french-both.pkl')
train = load_clean_sentences('english-french-train.pkl')
test = load_clean_sentences('english-french-test.pkl')


In [ ]:
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))


In [ ]:
# prepare french tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
print('French Vocabulary Size: %d' % ger_vocab_size)
print('French Max Length: %d' % (ger_length))


In [ ]:
# prepare training data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = encode_output(trainY, eng_vocab_size)


In [ ]:
# prepare validation data
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])
testY = encode_sequences(eng_tokenizer, eng_length, test[:, 0])
testY = encode_output(testY, eng_vocab_size)


In [ ]:
# define model
model = define_model(ger_vocab_size, eng_vocab_size, ger_length, eng_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')
# summarize defined model
print(model.summary())
plot_model(model, to_file='model.png', show_shapes=True)
# fit model
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=30, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)


In [ ]:

# In[8]:


In [ ]:

from pickle import load
from numpy import array, argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu


In [ ]:
def load_dataset(filename):
    return load(open(filename, 'rb'))


In [ ]:
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer


In [ ]:
# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)


In [ ]:
# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X


In [ ]:
# Map an integer to a word
def map_int_to_word(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None


In [ ]:
# Predict the target sequence
def predict_sequence(model, tokenizer, source):
    pred = model.predict(source, verbose=0)[0]
    integers = [argmax(vector) for vector in pred]
    target = list()
    for i in integers:
        word = map_int_to_word(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)


In [ ]:
# Evaluate the model
def evaluate_model(model, tokenizer, source, raw_dataset):
    predicted, actual = list(), list()
    for i, source in enumerate(source):
        source = source.reshape((1, source.shape[0]))
        translation = predict_sequence(model, tokenizer, source)
        raw_target= raw_dataset[i]
        if i < 10:
            # print('src=[%s], target=[%s], predicted=[%s]' % (raw_source, raw_target, translation))
          actual.append(' '.join(list(raw_target)).split())
          predicted.append(translation.split())


In [ ]:
    # Bleu Scores
    print('Bleu-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('Bleu-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('Bleu-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('Bleu-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))


In [ ]:
# Load datasets
dataset = load_dataset('english-french-both.pkl')
train = load_dataset('english-french-train.pkl')
test = load_dataset('english-french-test.pkl')


In [ ]:
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])


In [ ]:
# prepare french tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])


In [ ]:
# Prepare data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])


In [ ]:
model = load_model('model.h5')


In [ ]:
print('Testing on trained examples')
evaluate_model(model, eng_tokenizer, trainX, train)


In [ ]:
print('Testing on test examples')
evaluate_model(model, eng_tokenizer, testX, test)


In [ ]:

# In[ ]:
